# Computational Theory Assessment
---

In [1]:
# Import necessary libraries
import numpy as np

## Problem 1: Binary Words and Operations

### Implement the following functions in Python. Use **NumPy** to ensure that all variables and values are treated as **32-bit integers**. These functions are defined in the **Secure Hash Standard (see page 10)**.

1. `Parity(x, y, z)`
2. `Ch(x, y, z)`
3. `Maj(x, y, z)`
4. `Sigma0(x)` - *written as $\Sigma_0^{256}(x)$ in the standard.*
5. `Sigma1(x)` - *written as $\Sigma_1^{256}(x)$ in the standard.*
6. `sigma0(x)` - *written as $\sigma_0^{256}(x)$ in the standard.*
7. `sigma1(x)` - *written as $\sigma_1^{256}(x)$ in the standard.*

### Instructions

- Document each function with a clear docstring
- Explain its purpose and behaviour in Markdown
- Test it with appropriate examples to verify correctness.

### Solution:

**1. SHA-1 Parity(x, y, z) Function**

The Parity function is a linear bitwise operation used in certain rounds of the SHA-1 hash algorithm (specifically rounds 20-39 and 60-79). It takes three 32-bit integers, which we can call `x`, `y`, and `z`, and combines them by applying the XOR operation to each corresponding bit. 

$$
Parity(x, y, z) = x \oplus y \oplus z
$$

XOR (exclusive OR) works like this: for each bit position, if the number of `1`s in that position across `x`, `y`, and `z` is **odd**, the result bit is `1`. If it is **even**, then the result bit is `0`. 

This function helps increase **diffusion** in the hash algorithm, meaning that small changes in input spread out and affect many bits in the output. This contributes to the security of SHA-1 by making it more resistant to certain types of cryptographic attacks.

In [2]:
def Parity(x, y, z):
    """
    Performs bitwise XOR across three 32-bit integers.

    This function is used in the SHA-1 algorithm during rounds 20-39 and 60-79.

    Parameters:
        - x (int): The first 32-bit integer.
        - y (int): The second 32-bit integer.
        - z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of x XOR y XOR z, as a 32-bit unsigned integer.
    """

    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return x ^ y ^ z # Bitwise XOR operation

In [3]:
# Test cases to validate the Parity function
def test_parity():
    # Basic tests
    assert Parity(0b0000, 0b0000, 0b0000) == 0b0000, "All zeros"
    assert Parity(0b0001, 0b0000, 0b0000) == 0b0001, "Single one"
    assert Parity(0b0001, 0b0001, 0b0000) == 0b0000, "Two ones"
    assert Parity(0b0001, 0b0001, 0b0001) == 0b0001, "Three ones"
    
    # Bit pattern tests
    assert Parity(0b1010, 0b0101, 0b1111) == 0b0000, "All bits toggle"
    assert Parity(0b1100, 0b0011, 0b0101) == 0b1010, "Mixed bit patterns"
    assert Parity(0b1111, 0b1111, 0b1111) == 0b1111, "All ones in small value"
    
    # Full 32-bit value tests
    # Switched to hex notation for better readability with 32-bit values
    assert Parity(0xFFFFFFFF, 0x00000000, 0x00000000) == 0xFFFFFFFF, "One max value"
    assert Parity(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) == 0x00000000, "Two max values"
    assert Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) == 0xFFFFFFFF, "All max values"
    # 0xAAAAAAAA represents 10101010... , 0x55555555 represents 01010101...
    assert Parity(0xAAAAAAAA, 0x55555555, 0xFFFFFFFF) == 0x00000000, "Alternating bits pattern" 
                                                                                                  
    
    # Edge cases
    assert Parity(0x00000000, 0x00000000, 0x00000000) == 0x00000000, "All min values"
    assert Parity(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF) == 0x00000000, "Two identical, one different"
    
    # This final test uses complex hex values and explicit type conversion to verify
    # the function works correctly with arbitrary inputs under the same conditions
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF # Non-repeating patterns and common test values
    x_uint, y_uint, z_uint = np.uint32(x), np.uint32(y), np.uint32(z)
    expected = x_uint ^ y_uint ^ z_uint
    assert Parity(x, y, z) == expected, "Complex values don't match formula"
    
    print("All Parity function test cases passed.")

# Run the tests
test_parity()

All Parity function test cases passed.


**2. Ch(x, y, z) Function**

The Ch (Choice) function is a nonlinear bitwise operation used in both SHA-1 (specifically in rounds 0-19) and SHA-256 hash algorithms. It takes three 32-bit integers as input: `x`, `y`, and `z`.

$$
Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

The Ch function operates as a bit selector or multiplexer:
- For each bit position, if the bit in `x` is `1`, the function selects the corresponding bit from `y`
- If the bit in `x` is `0`, the function selects the corresponding bit from `z`

This provides important **nonlinear mixing** in the hash algorithm. Unlike the Parity function which is linear, the Ch function has nonlinear behavior that strengthens the cryptographic properties of SHA algorithms by making them more resistant to differential cryptanalysis and other attacks.

The name "Ch" reflects its behavior as a "chooser" or "selector" between the bits of `y` and `z` based on `x`.

In [4]:
def Ch(x, y, z):
    """
    Performs the Choice function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 0-19
    - SHA-256 algorithm 
    
    Parameters:
        - x (int): The first 32-bit integer (selector)
        - y (int): The second 32-bit integer (selected when x bit is 1)
        - z (int): The third 32-bit integer (selected when x bit is 0)
        
    Returns:
        np.uint32: The result of (x & y) ^ (~x & z), as a 32-bit unsigned integer
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Calculate and return the Choice function
    return (x & y) ^ (~x & z)  # Bitwise operations for bit selection

In [5]:
# Test cases to validate the Ch function
def test_ch():
    # Basic tests
    assert Ch(0b0000, 0b0000, 0b0000) == 0b0000, "All zeros"  # When x=0, select z (which is 0)
    assert Ch(0b1111, 0b0000, 0b0000) == 0b0000, "x=1, both y and z are 0"  # When x=1, select y (which is 0)
    assert Ch(0b0000, 0b1111, 0b0000) == 0b0000, "x=0, select z (which is 0)"  # When x=0, select z, ignoring y
    assert Ch(0b0000, 0b0000, 0b1111) == 0b1111, "x=0, select z (which is 1)"  # When x=0, select z (which is 1)
    
    # Bit selection tests
    assert Ch(0b1010, 0b1100, 0b0011) == 0b1001, "Alternating selection"  # x selects alternating bits from y and z
    assert Ch(0b1100, 0b1111, 0b0000) == 0b1100, "Mixed selection pattern"  # x=1 selects y bits, x=0 selects z bits
    
    # Full 32-bit value tests
    # Switched to hex notation for better readability with 32-bit values
    assert Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555) == 0xAAAAAAAA, "When x=all 1s, select y"  # x=1 means select all bits from y
    assert Ch(0x00000000, 0xAAAAAAAA, 0x55555555) == 0x55555555, "When x=all 0s, select z"  # x=0 means select all bits from z
    assert Ch(0xAAAAAAAA, 0xFFFFFFFF, 0x00000000) == 0xAAAAAAAA, "x=10101..., y=all 1s, z=all 0s"  # Select y when x=1, z when x=0
    
    # Edge cases
    assert Ch(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) == 0xFFFFFFFF, "x and y are max values"  # When x=1, select all from y
    assert Ch(0x00000000, 0xFFFFFFFF, 0x00000000) == 0x00000000, "x=0, z=0"  # When x=0, select all from z (which is 0)
    
    # This final test uses complex hex values and explicit type conversion to verify
    # the function works correctly with arbitrary inputs under the same conditions
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF  # Non-repeating patterns and common test values
    x_uint, y_uint, z_uint = np.uint32(x), np.uint32(y), np.uint32(z)  
    expected = (x_uint & y_uint) ^ (~x_uint & z_uint)  
    assert Ch(x, y, z) == expected, "Complex values don't match formula"  
    
    print("All Ch function test cases passed.")

# Run the tests
test_ch()

All Ch function test cases passed.


**3. Maj(x, y, z) Function**

The Maj (Majority) function is a nonlinear bitwise operation used in both SHA-1 (specifically rounds 40-59) and SHA-256 hash algorithms. It takes three 32-bit integers as input: `x`, `y`, and `z`.

$$
Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$

The Maj function implements a bit-wise majority vote: for each bit position, the output bit is `1` if at least two of the three input bits are `1`, otherwise it is `0`.

This function provides nonlinear mixing in the hash algorithm. The majority function is particularly effective at creating avalanche effects where small changes in input bits propagate through the algorithm in complex ways. This property is crucial for the cryptographic strength of SHA algorithms.

The name "Maj" reflects its behavior as a "majority" selector that chooses the most common bit value among the three inputs at each position.


In [6]:
def Maj(x, y, z):
    """
    Performs the Majority function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 40-59
    - SHA-256 algorithm
    
    Parameters:
        - x (int): The first 32-bit integer
        - y (int): The second 32-bit integer
        - z (int): The third 32-bit integer
        
    Returns:
        np.uint32: The result of (x & y) ^ (x & z) ^ (y & z), as a 32-bit unsigned integer
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Calculate and return the Majority function
    return (x & y) ^ (x & z) ^ (y & z)  # Bitwise operations for majority selection

---

## Problem 2: Fractional Parts of Cube Roots

### Use **NumPy** to calculate the constants listed at the bottom of **page 11 of the Secure Hash Standard**, following the steps below. These are the **first 32 bits of the fractional parts** of the **cube roots of the first 64 prime numbers**.

### Instructions

- Write a function called `primes(n)` that generates the first `n` prime numbers.
- Use the function to calculate the **cube root** of the first 64 primes.
- For each cube root, extract the first thirty-two bits of the fractional part.
- Display the result in **hexadecimal**.
- Test the results against what is in the Secure Hash Standard.

### Solution:

---

## Problem 3: Padding

### Instructions

- Write a generator function `block_parse(msg)` that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 
- The function should accept a bytes object called `msg`. 
- At each iteration, it should yield the next 512-bit block of `msg` as a bytes object. 
- Ensure that the final block (or final two blocks) include the required padding of `msg` as specified in the standard. 
- Test the generator with messages of different lengths to confirm proper padding and block output.

### Solution:

---

## Problem 4: Hashes

### Instructions

- Write a function `hash(current, block)` that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

### Solution:

---

## Problem 5: Passwords

### Instructions

- The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. 
- As strings, they were encoded using UTF-8. 
- Determine the passwords and explain how you found them. 
- Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

### SHA-256 hashes 
1. 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
2. 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
3. b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

### Solution:

---
# End of Assessment